### Software Vulnerability Assessment Agent

**Task:** Transform the Red Team agent concept into a comprehensive software vulnerability scanner that can analyze your codebase for security weaknesses.

> **Key Insight:** Instead of just testing LLM prompt injection, we'll create an AI-powered security audit tool that can identify common software vulnerabilities based on OWASP Top 10, CWE (Common Weakness Enumeration), and security best practices.

**What We'll Check For:**
- **Injection Vulnerabilities**: SQL injection, command injection, LDAP injection
- **Authentication & Authorization Flaws**: Weak password policies, insecure session management
- **Sensitive Data Exposure**: Hardcoded secrets, unencrypted data storage
- **XML External Entities (XXE)**: Unsafe XML parsing
- **Broken Access Control**: Missing authorization checks
- **Security Misconfiguration**: Default credentials, verbose error messages
- **Cross-Site Scripting (XSS)**: Reflected, stored, and DOM-based XSS
- **Insecure Deserialization**: Unsafe object deserialization
- **Using Components with Known Vulnerabilities**: Outdated dependencies
- **Insufficient Logging & Monitoring**: Missing security event logging

**Instructions:**
1. Create a vulnerability assessment agent that can analyze Python code files
2. Generate specific test cases and scanning patterns for each vulnerability type
3. Scan the actual codebase files in the workspace
4. Provide detailed reports with severity levels and remediation suggestions

In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Verify the utils directory exists
utils_path = os.path.join(project_root, 'utils')
print(f"Utils path exists: {os.path.exists(utils_path)}")
print(f"Project root: {project_root}")

# Try to import utils and setup LLM client
client, model_name, api_provider = None, None, None

try:
    from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output
    print("Successfully imported utils functions!")
    
    # Setup the LLM client
    try:
        client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")
        if client and model_name and api_provider:
            print(f"✅ LLM client initialized: {model_name} via {api_provider}")
        else:
            print("⚠️ LLM client initialization returned None values")
            raise Exception("Client initialization failed")
    except Exception as llm_error:
        print(f"⚠️ LLM setup failed: {llm_error}")
        client, model_name, api_provider = None, "gpt-4o", "openai"
        print("Using fallback configuration - AI analysis will be disabled")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print(f"sys.path: {sys.path}")
    
    # Create fallback functions
    def get_completion(prompt, client, model, provider):
        """Dummy function - AI analysis will be skipped"""
        return '{"vulnerabilities_found": [], "overall_security_score": "5", "summary": "AI analysis not available - missing LLM setup"}'
    
    def save_artifact(content, path, overwrite=False):
        """Dummy save function"""
        print(f"Would save to: {path}")
        return True
    
    def load_artifact(path):
        """Dummy load function"""
        print(f"Would load from: {path}")
        return None
    
    def clean_llm_output(content, language='python'):
        """Dummy clean function"""
        return content
    
    client, model_name, api_provider = None, "gpt-4o", "openai"
    print("Using basic fallback configuration - AI features disabled")

print(f"Final configuration: client={client is not None}, model={model_name}, provider={api_provider}")

2025-10-03 10:35:56,704 ag_aisoftdev.utils WARNING Optional core dependencies not found. Some features will be degraded. provider=None model=None latency_ms=None artifacts_path=None
2025-10-03 10:35:56,705 ag_aisoftdev.utils WARNING To enable full functionality run: pip install python-dotenv ipython plantuml provider=None model=None latency_ms=None artifacts_path=None
2025-10-03 10:35:56,705 ag_aisoftdev.utils WARNING To enable full functionality run: pip install python-dotenv ipython plantuml provider=None model=None latency_ms=None artifacts_path=None
2025-10-03 10:35:56,713 ag_aisoftdev.utils WARNING 'requests' package not installed; HTTP helpers are disabled. provider=None model=None latency_ms=None artifacts_path=None
2025-10-03 10:35:56,713 ag_aisoftdev.utils WARNING 'requests' package not installed; HTTP helpers are disabled. provider=None model=None latency_ms=None artifacts_path=None
2025-10-03 10:35:56,722 ag_aisoftdev.utils WARNING python-dotenv not installed; .env will not 

Utils path exists: True
Project root: c:\Users\labadmin\Documents\repo\KMSH_contracting
Successfully imported utils functions!
⚠️ LLM client initialization returned None values
⚠️ LLM setup failed: Client initialization failed
Using fallback configuration - AI analysis will be disabled
Final configuration: client=False, model=gpt-4o, provider=openai


In [6]:
# Alternative: Basic setup without utils (if utils import fails)
# Uncomment and use this if the above setup doesn't work

# client = None
# model_name = "gpt-4o" 
# api_provider = "openai"
# project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# def get_completion(prompt, client, model, provider):
#     """Dummy function - AI analysis will be skipped"""
#     return '{"vulnerabilities_found": [], "overall_security_score": "5", "summary": "AI analysis not available"}'

# print("Using basic setup - AI analysis features will be limited")

In [2]:
import os
import re
from pathlib import Path
from typing import List, Dict, Tuple
import json

# Software Vulnerability Assessment Agent
class SoftwareVulnerabilityScanner:
    def __init__(self, client, model_name, api_provider):
        self.client = client
        self.model_name = model_name
        self.api_provider = api_provider
        self.ai_enabled = client is not None and 'get_completion' in globals()
        
        if self.ai_enabled:
            print("🤖 AI-powered analysis: ENABLED")
        else:
            print("🔍 Pattern-based scanning only: AI analysis DISABLED")
        
        # Define vulnerability patterns and their severity levels
        self.vulnerability_patterns = {
            "sql_injection": {
                "patterns": [
                    r"execute\s*\(\s*[\"'][^\"']*\+.*[\"']\s*\)",
                    r"cursor\.execute\s*\(\s*[\"'][^\"']*%.*[\"']\s*%",
                    r"\.format\s*\([^)]*\)\s*\)",
                    r"f[\"'][^\"']*\{[^}]*\}[^\"']*[\"']"
                ],
                "severity": "HIGH",
                "description": "Potential SQL Injection vulnerability"
            },
            "hardcoded_secrets": {
                "patterns": [
                    r"password\s*=\s*[\"'][^\"']{8,}[\"']",
                    r"api_key\s*=\s*[\"'][^\"']{20,}[\"']",
                    r"secret\s*=\s*[\"'][^\"']{10,}[\"']",
                    r"token\s*=\s*[\"'][^\"']{20,}[\"']"
                ],
                "severity": "CRITICAL",
                "description": "Hardcoded secrets or credentials found"
            },
            "command_injection": {
                "patterns": [
                    r"os\.system\s*\([^)]*\+",
                    r"subprocess\.(call|run|Popen)\s*\([^)]*\+",
                    r"eval\s*\(",
                    r"exec\s*\("
                ],
                "severity": "HIGH",
                "description": "Potential command injection vulnerability"
            },
            "insecure_random": {
                "patterns": [
                    r"random\.random\(\)",
                    r"random\.randint\(",
                    r"random\.choice\("
                ],
                "severity": "MEDIUM",
                "description": "Use of insecure random number generator for security purposes"
            },
            "weak_crypto": {
                "patterns": [
                    r"hashlib\.md5\(",
                    r"hashlib\.sha1\(",
                    r"DES\.",
                    r"RC4\."
                ],
                "severity": "MEDIUM",
                "description": "Use of weak cryptographic algorithms"
            },
            "debug_mode": {
                "patterns": [
                    r"debug\s*=\s*True",
                    r"DEBUG\s*=\s*True",
                    r"app\.debug\s*=\s*True"
                ],
                "severity": "MEDIUM",
                "description": "Debug mode enabled - may expose sensitive information"
            },
            "unsafe_pickle": {
                "patterns": [
                    r"pickle\.loads?\s*\(",
                    r"cPickle\.loads?\s*\("
                ],
                "severity": "HIGH",
                "description": "Unsafe deserialization with pickle - can lead to code execution"
            }
        }

    def scan_file(self, file_path: str) -> List[Dict]:
        """Scan a single file for vulnerabilities"""
        vulnerabilities = []
        
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                lines = content.split('\n')
                
            for vuln_type, config in self.vulnerability_patterns.items():
                for pattern in config['patterns']:
                    matches = re.finditer(pattern, content, re.IGNORECASE)
                    for match in matches:
                        # Find line number
                        line_num = content[:match.start()].count('\n') + 1
                        line_content = lines[line_num - 1].strip()
                        
                        vulnerabilities.append({
                            'type': vuln_type,
                            'severity': config['severity'],
                            'description': config['description'],
                            'file': file_path,
                            'line': line_num,
                            'code': line_content,
                            'match': match.group()
                        })
        except Exception as e:
            print(f"Error scanning {file_path}: {e}")
            
        return vulnerabilities

    def ai_code_review(self, file_path: str, code_snippet: str) -> Dict:
        """Use AI to perform deeper security analysis"""
        if not self.ai_enabled:
            return {"vulnerabilities_found": [], "overall_security_score": "5", "summary": "AI analysis disabled"}
            
        prompt = f"""
        You are a cybersecurity expert conducting a security code review. Analyze the following Python code for security vulnerabilities.

        File: {file_path}
        Code:
        ```python
        {code_snippet}
        ```

        Look for:
        1. Injection vulnerabilities (SQL, command, LDAP)
        2. Authentication and authorization flaws
        3. Sensitive data exposure
        4. Security misconfigurations
        5. Input validation issues
        6. Cryptographic weaknesses
        7. Error handling that reveals sensitive information

        Respond with a JSON object containing:
        {{
            "vulnerabilities_found": [
                {{
                    "type": "vulnerability_type",
                    "severity": "CRITICAL|HIGH|MEDIUM|LOW",
                    "description": "detailed description",
                    "line_numbers": [1, 2, 3],
                    "recommendation": "how to fix this issue"
                }}
            ],
            "overall_security_score": "1-10 where 10 is most secure",
            "summary": "brief security assessment summary"
        }}
        """
        
        try:
            response = get_completion(prompt, self.client, self.model_name, self.api_provider)
            # Extract JSON from response
            json_match = re.search(r'\{.*\}', response, re.DOTALL)
            if json_match:
                return json.loads(json_match.group(0))
        except Exception as e:
            print(f"AI analysis error: {e}")
            
        return {"vulnerabilities_found": [], "overall_security_score": "0", "summary": "Analysis failed"}

    def scan_codebase(self, directory: str = None) -> Dict:
        """Scan entire codebase for vulnerabilities"""
        if directory is None:
            directory = project_root
            
        results = {
            "scanned_files": 0,
            "total_vulnerabilities": 0,
            "vulnerabilities_by_severity": {"CRITICAL": 0, "HIGH": 0, "MEDIUM": 0, "LOW": 0},
            "detailed_findings": []
        }
        
        print(f"🔍 Scanning directory: {directory}")
        
        # Scan Python files
        for py_file in Path(directory).rglob("*.py"):
            if "test" in str(py_file).lower() or "__pycache__" in str(py_file):
                continue
                
            results["scanned_files"] += 1
            print(f"  📄 Scanning: {os.path.basename(py_file)}")
            file_vulns = self.scan_file(str(py_file))
            
            # Add AI-powered analysis for files with initial findings (only if AI is enabled)
            if file_vulns and self.ai_enabled:
                try:
                    with open(py_file, 'r', encoding='utf-8') as f:
                        code_content = f.read()
                    
                    # Limit code size for AI analysis
                    if len(code_content) < 5000:  # Only analyze smaller files
                        print(f"    🤖 Running AI analysis on {os.path.basename(py_file)}...")
                        ai_analysis = self.ai_code_review(str(py_file), code_content)
                        
                        # Merge AI findings
                        for ai_vuln in ai_analysis.get("vulnerabilities_found", []):
                            file_vulns.append({
                                'type': f"ai_detected_{ai_vuln['type']}",
                                'severity': ai_vuln['severity'],
                                'description': ai_vuln['description'],
                                'file': str(py_file),
                                'line': ai_vuln.get('line_numbers', [0])[0],
                                'code': 'AI-detected vulnerability',
                                'recommendation': ai_vuln.get('recommendation', 'No recommendation provided')
                            })
                except Exception as e:
                    print(f"    ❌ AI analysis failed for {py_file}: {e}")
            
            for vuln in file_vulns:
                results["total_vulnerabilities"] += 1
                results["vulnerabilities_by_severity"][vuln["severity"]] += 1
                results["detailed_findings"].append(vuln)
        
        return results

# Check if we have the required variables
missing_vars = []
for var_name in ['client', 'model_name', 'api_provider']:
    if var_name not in globals():
        missing_vars.append(var_name)

if missing_vars:
    print(f"⚠️  Missing variables: {missing_vars}")
    print("Using default values...")
    if 'client' not in globals():
        client = None
    if 'model_name' not in globals():
        model_name = "gpt-4o"
    if 'api_provider' not in globals():
        api_provider = "openai"

# Initialize the vulnerability scanner
vuln_scanner = SoftwareVulnerabilityScanner(client, model_name, api_provider)
print("✅ Software Vulnerability Scanner initialized!")

🔍 Pattern-based scanning only: AI analysis DISABLED
✅ Software Vulnerability Scanner initialized!


In [3]:
# Run the comprehensive vulnerability scan
print("🔍 Starting comprehensive software vulnerability assessment...")
print("=" * 60)

# Ensure project_root is defined (in case setup cell wasn't run properly)
if 'project_root' not in globals():
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
    print(f"Setting project_root to: {project_root}")

# Scan the utils directory (where most of the Python code is)
utils_dir = os.path.join(project_root, "utils")
print(f"Scanning directory: {utils_dir}")
print(f"Directory exists: {os.path.exists(utils_dir)}")

scan_results = vuln_scanner.scan_codebase(utils_dir)

print(f"📊 SCAN SUMMARY")
print(f"Files scanned: {scan_results['scanned_files']}")
print(f"Total vulnerabilities found: {scan_results['total_vulnerabilities']}")
print("\n🚨 VULNERABILITIES BY SEVERITY:")
for severity, count in scan_results['vulnerabilities_by_severity'].items():
    if count > 0:
        print(f"  {severity}: {count}")

print("\n" + "=" * 60)
print("📋 DETAILED FINDINGS:")
print("=" * 60)

# Group findings by file for better readability
findings_by_file = {}
for finding in scan_results['detailed_findings']:
    file_path = finding['file']
    if file_path not in findings_by_file:
        findings_by_file[file_path] = []
    findings_by_file[file_path].append(finding)

# Display findings
for file_path, findings in findings_by_file.items():
    print(f"\n📁 FILE: {os.path.basename(file_path)}")
    print("-" * 40)
    
    for i, finding in enumerate(findings, 1):
        severity_icon = {"CRITICAL": "🔴", "HIGH": "🟠", "MEDIUM": "🟡", "LOW": "🟢"}
        icon = severity_icon.get(finding['severity'], "⚪")
        
        print(f"{i}. {icon} {finding['severity']} - {finding['type'].upper()}")
        print(f"   📍 Line {finding['line']}: {finding['description']}")
        print(f"   📝 Code: {finding['code'][:100]}{'...' if len(finding['code']) > 100 else ''}")
        if 'recommendation' in finding:
            print(f"   💡 Fix: {finding['recommendation']}")
        print()

print("=" * 60)
print("🎯 SECURITY RECOMMENDATIONS:")
print("=" * 60)

# Generate overall security recommendations
if scan_results['total_vulnerabilities'] > 0:
    print(f"""
🔒 IMMEDIATE ACTIONS REQUIRED:
1. Address all CRITICAL vulnerabilities immediately
2. Review and fix HIGH severity issues within 24-48 hours
3. Plan remediation for MEDIUM severity issues
4. Consider LOW severity issues for future security hardening

🛡️ SECURITY BEST PRACTICES:
1. Use parameterized queries to prevent SQL injection
2. Store secrets in environment variables, not hardcoded
3. Use secure random generators from the 'secrets' module
4. Implement proper input validation and sanitization
5. Use strong cryptographic algorithms (SHA-256, AES-256)
6. Add security headers and proper error handling
7. Implement comprehensive logging for security events
""")
else:
    print("✅ No obvious vulnerabilities detected in the scanned code!")
    print("🔍 Consider running additional security tests and penetration testing")

print("\n" + "=" * 60)
print("✅ Vulnerability assessment complete!")

🔍 Starting comprehensive software vulnerability assessment...
Scanning directory: c:\Users\labadmin\Documents\repo\KMSH_contracting\utils
Directory exists: True
🔍 Scanning directory: c:\Users\labadmin\Documents\repo\KMSH_contracting\utils
  📄 Scanning: artifacts.py
  📄 Scanning: audio.py
  📄 Scanning: errors.py
  📄 Scanning: helpers.py
  📄 Scanning: http.py
  📄 Scanning: image_gen.py
  📄 Scanning: llm.py
  📄 Scanning: logging.py
  📄 Scanning: models.py
  📄 Scanning: plantuml.py
  📄 Scanning: rate_limit.py
  📄 Scanning: settings.py
  📄 Scanning: __init__.py
  📄 Scanning: anthropic.py
  📄 Scanning: base.py
  📄 Scanning: google.py
  📄 Scanning: huggingface.py
  📄 Scanning: openai.py
  📄 Scanning: __init__.py
📊 SCAN SUMMARY
Files scanned: 19
Total vulnerabilities found: 31

🚨 VULNERABILITIES BY SEVERITY:
  HIGH: 31

📋 DETAILED FINDINGS:

📁 FILE: artifacts.py
----------------------------------------
1. 🟠 HIGH - SQL_INJECTION
   📍 Line 138: Potential SQL Injection vulnerability
   📝 Code: ra

In [4]:
# Advanced Dependency and Configuration Vulnerability Scanner
class AdvancedSecurityScanner:
    def __init__(self, client, model_name, api_provider):
        self.client = client
        self.model_name = model_name
        self.api_provider = api_provider
    
    def scan_dependencies(self, requirements_file: str = None) -> Dict:
        """Scan dependencies for known vulnerabilities"""
        if requirements_file is None:
            requirements_file = os.path.join(project_root, "requirements.txt")
        
        vulnerabilities = []
        
        try:
            with open(requirements_file, 'r') as f:
                dependencies = f.readlines()
            
            # AI-powered dependency analysis
            deps_text = '\n'.join(dependencies)
            prompt = f"""
            You are a cybersecurity expert analyzing Python dependencies for known vulnerabilities.
            
            Dependencies:
            {deps_text}
            
            For each dependency, check if there are known security vulnerabilities or if the version is outdated.
            Consider:
            1. Known CVEs (Common Vulnerabilities and Exposures)
            2. Outdated versions with security patches available
            3. Dependencies with history of security issues
            4. Abandoned or unmaintained packages
            
            Respond with JSON:
            {{
                "vulnerable_packages": [
                    {{
                        "package": "package_name",
                        "current_version": "version",
                        "vulnerability": "CVE-2023-xxxx or description",
                        "severity": "CRITICAL|HIGH|MEDIUM|LOW",
                        "recommended_action": "update to version X.X.X or replace with alternative"
                    }}
                ],
                "security_score": "1-10",
                "recommendations": ["list of general recommendations"]
            }}
            """
            
            response = get_completion(prompt, self.client, self.model_name, self.api_provider)
            json_match = re.search(r'\{.*\}', response, re.DOTALL)
            if json_match:
                return json.loads(json_match.group(0))
                
        except Exception as e:
            print(f"Error scanning dependencies: {e}")
        
        return {"vulnerable_packages": [], "security_score": "5", "recommendations": []}
    
    def scan_configuration_files(self) -> List[Dict]:
        """Scan configuration files for security issues"""
        config_vulns = []
        
        # Check common configuration files
        config_files = [
            "pyproject.toml",
            "requirements.txt", 
            ".env",
            "config.py",
            "settings.py"
        ]
        
        for config_file in config_files:
            file_path = os.path.join(project_root, config_file)
            if os.path.exists(file_path):
                try:
                    with open(file_path, 'r') as f:
                        content = f.read()
                    
                    # AI analysis of configuration
                    prompt = f"""
                    Analyze this configuration file for security issues:
                    
                    File: {config_file}
                    Content:
                    {content[:2000]}  # Limit content size
                    
                    Look for:
                    1. Hardcoded credentials or API keys
                    2. Insecure default settings
                    3. Debug mode enabled in production
                    4. Overly permissive configurations
                    5. Missing security headers or settings
                    
                    Respond with JSON:
                    {{
                        "issues": [
                            {{
                                "type": "issue_type",
                                "severity": "CRITICAL|HIGH|MEDIUM|LOW",
                                "description": "description",
                                "recommendation": "how to fix"
                            }}
                        ]
                    }}
                    """
                    
                    response = get_completion(prompt, self.client, self.model_name, self.api_provider)
                    json_match = re.search(r'\{.*\}', response, re.DOTALL)
                    if json_match:
                        analysis = json.loads(json_match.group(0))
                        for issue in analysis.get('issues', []):
                            issue['file'] = config_file
                            config_vulns.append(issue)
                            
                except Exception as e:
                    print(f"Error analyzing {config_file}: {e}")
        
        return config_vulns

# Ensure project_root is defined (in case setup cell wasn't run properly)
if 'project_root' not in globals():
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
    print(f"Setting project_root to: {project_root}")

# Run advanced security scanning
advanced_scanner = AdvancedSecurityScanner(client, model_name, api_provider)

print("\n🔍 ADVANCED SECURITY ANALYSIS")
print("=" * 60)

# Dependency vulnerability scan
print("📦 Scanning dependencies for known vulnerabilities...")
dep_results = advanced_scanner.scan_dependencies()

if dep_results['vulnerable_packages']:
    print(f"\n🚨 VULNERABLE DEPENDENCIES FOUND:")
    for vuln in dep_results['vulnerable_packages']:
        severity_icon = {"CRITICAL": "🔴", "HIGH": "🟠", "MEDIUM": "🟡", "LOW": "🟢"}
        icon = severity_icon.get(vuln['severity'], "⚪")
        print(f"{icon} {vuln['package']} ({vuln.get('current_version', 'unknown version')})")
        print(f"   🐛 Issue: {vuln['vulnerability']}")
        print(f"   💡 Action: {vuln['recommended_action']}")
        print()
else:
    print("✅ No obvious dependency vulnerabilities detected")

print(f"📊 Dependency Security Score: {dep_results['security_score']}/10")

# Configuration file analysis
print("\n⚙️ Scanning configuration files...")
config_vulns = advanced_scanner.scan_configuration_files()

if config_vulns:
    print(f"\n🚨 CONFIGURATION ISSUES FOUND:")
    for issue in config_vulns:
        severity_icon = {"CRITICAL": "🔴", "HIGH": "🟠", "MEDIUM": "🟡", "LOW": "🟢"}
        icon = severity_icon.get(issue['severity'], "⚪")
        print(f"{icon} {issue['file']}: {issue['type']}")
        print(f"   📝 {issue['description']}")
        print(f"   💡 Fix: {issue['recommendation']}")
        print()
else:
    print("✅ No obvious configuration security issues detected")

print("\n📋 GENERAL SECURITY RECOMMENDATIONS:")
for rec in dep_results.get('recommendations', []):
    print(f"• {rec}")

print("\n" + "=" * 60)
print("🎯 NEXT STEPS FOR SECURITY HARDENING:")
print("=" * 60)
print("""
1. 🔄 Regular Updates: Set up automated dependency updates
2. 🛡️ Security Monitoring: Implement continuous security scanning
3. 🔐 Secrets Management: Use proper secret management tools
4. 📊 Security Metrics: Track security debt and improvements
5. 🧪 Penetration Testing: Conduct regular security assessments
6. 📚 Security Training: Keep team updated on security best practices
7. 🚨 Incident Response: Have a security incident response plan
""")


🔍 ADVANCED SECURITY ANALYSIS
📦 Scanning dependencies for known vulnerabilities...
Error scanning dependencies: [Errno 2] No such file or directory: 'c:\\Users\\labadmin\\Documents\\repo\\KMSH_contracting\\requirements.txt'
✅ No obvious dependency vulnerabilities detected
📊 Dependency Security Score: 5/10

⚙️ Scanning configuration files...
Error analyzing .env: [openai:gpt-4o] completion error: API client not initialized.
✅ No obvious configuration security issues detected

📋 GENERAL SECURITY RECOMMENDATIONS:

🎯 NEXT STEPS FOR SECURITY HARDENING:

1. 🔄 Regular Updates: Set up automated dependency updates
2. 🛡️ Security Monitoring: Implement continuous security scanning
3. 🔐 Secrets Management: Use proper secret management tools
4. 📊 Security Metrics: Track security debt and improvements
5. 🧪 Penetration Testing: Conduct regular security assessments
6. 📚 Security Training: Keep team updated on security best practices
7. 🚨 Incident Response: Have a security incident response plan



In [5]:
# Generate and Save Security Review Report
def generate_security_report(scan_results, dep_results=None, config_vulns=None, output_file="security_review.md"):
    """Generate a comprehensive security review report in Markdown format"""
    
    from datetime import datetime
    import os
    
    # Ensure project_root is available - use global if exists, otherwise create it
    global project_root
    if 'project_root' not in globals() or project_root is None:
        project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
        print(f"Setting project_root to: {project_root}")
    
    report_path = os.path.join(project_root, output_file)
    
    # Generate timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Start building the markdown report
    report = f"""# Security Vulnerability Assessment Report

**Generated:** {timestamp}  
**Scanner:** Software Vulnerability Assessment Agent  
**Project:** KMSH_contracting  

---

## Executive Summary

This report contains the results of an automated security vulnerability assessment performed on the KMSH_contracting codebase. The scan identified potential security weaknesses using pattern-based detection and security best practices.

### Scan Overview
- **Files Scanned:** {scan_results['scanned_files']}
- **Total Vulnerabilities Found:** {scan_results['total_vulnerabilities']}
- **Scan Type:** {"AI-Enhanced + Pattern-Based" if 'vuln_scanner' in globals() and vuln_scanner.ai_enabled else "Pattern-Based Only"}

### Vulnerability Breakdown by Severity
"""
    
    # Add severity breakdown
    for severity in ['CRITICAL', 'HIGH', 'MEDIUM', 'LOW']:
        count = scan_results['vulnerabilities_by_severity'][severity]
        if count > 0:
            icon = {'CRITICAL': '🔴', 'HIGH': '🟠', 'MEDIUM': '🟡', 'LOW': '🟢'}[severity]
            report += f"- **{severity}:** {count} {icon}\n"
    
    report += "\n---\n\n"
    
    # Detailed Findings
    if scan_results['detailed_findings']:
        report += "## Detailed Vulnerability Findings\n\n"
        
        # Group findings by file
        findings_by_file = {}
        for finding in scan_results['detailed_findings']:
            file_path = finding['file']
            if file_path not in findings_by_file:
                findings_by_file[file_path] = []
            findings_by_file[file_path].append(finding)
        
        # Generate report for each file
        for file_path, findings in findings_by_file.items():
            filename = os.path.basename(file_path)
            report += f"### 📁 {filename}\n\n"
            report += f"**File Path:** `{file_path}`\n\n"
            
            for i, finding in enumerate(findings, 1):
                severity_icon = {'CRITICAL': '🔴', 'HIGH': '🟠', 'MEDIUM': '🟡', 'LOW': '🟢'}
                icon = severity_icon.get(finding['severity'], '⚪')
                
                report += f"#### {i}. {icon} {finding['severity']} - {finding['type'].replace('_', ' ').title()}\n\n"
                report += f"**Description:** {finding['description']}\n\n"
                report += f"**Location:** Line {finding['line']}\n\n"
                report += f"**Code:**\n```python\n{finding['code']}\n```\n\n"
                
                if 'recommendation' in finding:
                    report += f"**Recommendation:** {finding['recommendation']}\n\n"
                
                if 'match' in finding:
                    report += f"**Pattern Match:** `{finding['match']}`\n\n"
                
                report += "---\n\n"
    
    else:
        report += "## ✅ No Code Vulnerabilities Found\n\nNo obvious vulnerabilities were detected in the scanned Python files using pattern-based detection.\n\n"
    
    # Dependency Analysis (if available)
    if dep_results and dep_results.get('vulnerable_packages'):
        report += "## 📦 Dependency Vulnerability Analysis\n\n"
        report += f"**Security Score:** {dep_results['security_score']}/10\n\n"
        
        for vuln in dep_results['vulnerable_packages']:
            severity_icon = {'CRITICAL': '🔴', 'HIGH': '🟠', 'MEDIUM': '🟡', 'LOW': '🟢'}
            icon = severity_icon.get(vuln['severity'], '⚪')
            
            report += f"### {icon} {vuln['package']} - {vuln['severity']}\n\n"
            report += f"**Current Version:** {vuln.get('current_version', 'Unknown')}\n\n"
            report += f"**Vulnerability:** {vuln['vulnerability']}\n\n"
            report += f"**Recommended Action:** {vuln['recommended_action']}\n\n"
            report += "---\n\n"
    
    # Configuration Issues (if available)
    if config_vulns:
        report += "## ⚙️ Configuration Security Issues\n\n"
        
        for issue in config_vulns:
            severity_icon = {'CRITICAL': '🔴', 'HIGH': '🟠', 'MEDIUM': '🟡', 'LOW': '🟢'}
            icon = severity_icon.get(issue['severity'], '⚪')
            
            report += f"### {icon} {issue['file']} - {issue['severity']}\n\n"
            report += f"**Issue Type:** {issue['type']}\n\n"
            report += f"**Description:** {issue['description']}\n\n"
            report += f"**Recommendation:** {issue['recommendation']}\n\n"
            report += "---\n\n"
    
    # Security Recommendations
    report += """## 🎯 Security Recommendations

### Immediate Actions Required
1. **Address CRITICAL vulnerabilities immediately** - These pose severe security risks
2. **Fix HIGH severity issues within 24-48 hours** - These are significant security concerns
3. **Plan remediation for MEDIUM severity issues** - Schedule fixes in upcoming sprints
4. **Consider LOW severity issues for future hardening** - Include in technical debt backlog

### Security Best Practices
1. **Input Validation:** Implement proper input validation and sanitization
2. **Secrets Management:** Store secrets in environment variables, not hardcoded
3. **Secure Coding:** Use parameterized queries to prevent SQL injection
4. **Cryptography:** Use strong cryptographic algorithms (SHA-256, AES-256)
5. **Random Generation:** Use secure random generators from the 'secrets' module
6. **Error Handling:** Implement proper error handling without exposing sensitive information
7. **Security Headers:** Add appropriate security headers to web responses
8. **Logging:** Implement comprehensive security event logging

### Ongoing Security Measures
1. **Regular Updates:** Set up automated dependency updates
2. **Security Monitoring:** Implement continuous security scanning
3. **Penetration Testing:** Conduct regular security assessments
4. **Security Training:** Keep development team updated on security best practices
5. **Incident Response:** Maintain a security incident response plan
6. **Code Reviews:** Include security considerations in code review process
7. **Security Metrics:** Track security debt and improvements over time

---

## Report Metadata

**Scanner Version:** Software Vulnerability Assessment Agent v1.0  
**Scan Method:** Pattern-based detection with regex matching  
**File Types:** Python (.py) files  
**Excluded:** Test files, __pycache__ directories  

*This report was generated automatically. Manual security review is recommended for comprehensive assessment.*
"""
    
    # Save the report
    try:
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write(report)
        print(f"✅ Security review report saved to: {report_path}")
        print(f"📄 Report contains {len(report.split())} words and {len(report.splitlines())} lines")
        return report_path
    except Exception as e:
        print(f"❌ Error saving report: {e}")
        return None

# Ensure project_root is defined globally before running the report generator
if 'project_root' not in globals():
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
    print(f"Setting global project_root to: {project_root}")

# Generate and save the security report  
print("📝 Generating comprehensive security review report...")

# Collect all scan data (use the results from previous scans)
if 'scan_results' in globals():
    # Get dependency and config results if they exist
    dep_data = dep_results if 'dep_results' in globals() else None
    config_data = config_vulns if 'config_vulns' in globals() else None
    
    # Generate the report
    report_file = generate_security_report(
        scan_results=scan_results,
        dep_results=dep_data,
        config_vulns=config_data,
        output_file="security_review.md"
    )
    
    if report_file:
        print(f"📊 Security review report successfully generated!")
        print(f"🔗 Location: {report_file}")
        print(f"📋 You can now review the comprehensive security assessment in Markdown format.")
    else:
        print("❌ Failed to generate security report")
else:
    print("⚠️ No scan results available. Please run the vulnerability scan first.")
    print("   Run the previous cells to generate scan_results, then run this cell again.")

📝 Generating comprehensive security review report...
✅ Security review report saved to: c:\Users\labadmin\Documents\repo\KMSH_contracting\security_review.md
📄 Report contains 1350 words and 570 lines
📊 Security review report successfully generated!
🔗 Location: c:\Users\labadmin\Documents\repo\KMSH_contracting\security_review.md
📋 You can now review the comprehensive security assessment in Markdown format.
